<a href="https://colab.research.google.com/github/juanpalominoh/Fundamentos-de-Econometria/blob/main/Tema_13_Ecuaciones_Simult%C3%A1neas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Ecuaciones Simultáneas**

In [ ]:
# Instalamos paquetes 
install.packages("foreign")  # Leer base de datos de Stata
install.packages("skimr")    # Realizar descripción estadística
install.packages("AER")      # Estimar MC2E
install.packages("memisc")   # Paquete para realizar tablas

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
# Cargamos librerias
library(foreign)
library(dplyr)
library(skimr)
library(AER)
library(memisc)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




## **Oferta y demanda laboral de mujeres trabajadoras**

**Oferta laboral de mujeres trabajadoras y casadas (horas trabajadas)**

\begin{gather*}
hours=\beta_{0} + \beta_{1}lwage + \beta_{2}educ+ \beta_{3}age + \beta_{4}kidslt6 + \beta_{5}nwifeinc + \epsilon_{1}
\end{gather*}

donde `kidslt6` es el número de hijos que tienen menos de seis años de edad y `nwifeinc`es el ingreso no salarial de la mujer (que incluye los ingresos de su cónyuge).

**Demanda laboral de mujeres trabajadoras y casadas (oferta salarial)**

\begin{gather*}
lwage = \alpha_{0} + \alpha_{1}hours + \alpha_{2}educ + \alpha_{3}exper + \alpha_{4}exper^{2} + \epsilon_{2}
\end{gather*}

donde `age`(edad en años), `kidslt6`(número de niños), `nwifeinc` (ingreso no salarial de la mujer) son determinantes de la oferta de trabajo (`hours`), pero no de la demanda de trabajo (salario pagado) y puede ser un instrumento para `lwage`.

Asimismo, `experience` es un determinante de `lwage` pero no de cuántas horas trabajan las mujeres y puede ser usado como un instrumento para `hours`.

In [ ]:
# Cargar base de datos
base <- read.dta("/content/MROZ.dta", convert.factors = FALSE)
base

In [ ]:
# Solo mujeres que trabajan y variables que interesan
base <- base %>%
  filter(inlf == 1) %>%
  dplyr::select(hours, lwage, educ, exper, expersq, age, kidslt6, nwifeinc)

base

In [ ]:
# Estadísticas descriptivas
skimr::skim(base)

── Data Summary ────────────────────────
                           Values
Name                       base  
Number of rows             428   
Number of columns          8     
_______________________          
Column type frequency:           
  numeric                  8     
________________________         
Group variables            None  

── Variable type: numeric ──────────────────────────────────────────────────────
  skim_variable n_missing complete_rate     mean      sd      p0     p25     p50
1 hours                 0             1 1303.    776.    12      610.    1366.  
2 lwage                 0             1    1.19    0.723 -2.05     0.817    1.25
3 educ                  0             1   12.7     2.29   5       12       12   
4 exper                 0             1   13.0     8.06   0        7       12   
5 expersq               0             1  235.    270.     0       49      144   
6 age                   0             1   42.0     7.72  30       35       42   
7 ki

### **Ecuaciones estructurales**

\begin{gather*}
hours=\beta_{0} + \beta_{1}lwage + \beta_{2}educ+ \beta_{3}age + \beta_{4}kidslt6 + \beta_{5}nwifeinc + \epsilon_{1}
\end{gather*}

\begin{gather*}
lwage = \alpha_{0} + \alpha_{1}hours + \alpha_{2}educ + \alpha_{3}exper + \alpha_{4}exper^{2} + \epsilon_{2}
\end{gather*}


*   $exper$ y $exper^{2}$ son instrumentos para `lwage` ($exper$ y $exper^{2}$ no están en la ecuación `hours`, pero sí están relacionados al `lwage`)

*   Lo mismo para `age`, `kidslt6` y `nwifeinc` que son buenos instrumentos para `hours`.





### **Estimación para la Oferta Laboral**

**MC2E, primera etapa**: forma reducida, regresión de variables endógenas en todas las variables exógenas y obtención de valores predichos.

**Primera etapa para la ecuación 1**: variable endógena `lwage` sobre instrumentos $exper$ y $exper^2$ y variables exógenas:
\begin{gather*}
lwage = \gamma_{0} + \gamma_{1}exper + \gamma_{2}exper^{2} + \gamma_{3}educ + \gamma_{4}age + \gamma_{5}kidslt6 + \gamma_{6}nwifeinc + \mu_{1}
\end{gather*}

In [ ]:
# MC2E Primera etapa
oferta_first <- lm(lwage ~ exper + expersq + educ + age + kidslt6 + nwifeinc, base)
summary(oferta_first)


Call:
lm(formula = lwage ~ exper + expersq + educ + age + kidslt6 + 
    nwifeinc, data = base)

Residuals:
     Min       1Q   Median       3Q      Max 
-3.09020 -0.30506  0.05005  0.37763  2.35628 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept) -0.4471607  0.2852028  -1.568  0.11766    
exper        0.0418643  0.0132377   3.162  0.00168 ** 
expersq     -0.0007625  0.0004008  -1.903  0.05779 .  
educ         0.1011113  0.0149618   6.758 4.68e-11 ***
age         -0.0025561  0.0051920  -0.492  0.62275    
kidslt6     -0.0532185  0.0884411  -0.602  0.54767    
nwifeinc     0.0055600  0.0033104   1.680  0.09378 .  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.6662 on 421 degrees of freedom
Multiple R-squared:  0.1633,	Adjusted R-squared:  0.1514 
F-statistic: 13.69 on 6 and 421 DF,  p-value: 3.158e-14


**MC2E, segunda etapa**: estimar las ecuaciones usando los valores predichos de la primera etapa para variables endógenas

\begin{gather*}
hours=\beta_{0} + \beta_{1}\hat{lwage} + \beta_{2}educ+ \beta_{3}age + \beta_{4}kidslt6 + \beta_{5}nwifeinc + \epsilon_{1}
\end{gather*}

In [ ]:
# Regresión para hours usando estimación MC2E
# lwage es instrumentalizada por variables de otra ecuación
oferta_second <- ivreg(hours ~ lwage + educ + age + kidslt6 + nwifeinc |
                  . - lwage + exper + expersq, data = base)

summary(oferta_second, diagnostics = TRUE)


Call:
ivreg(formula = hours ~ lwage + educ + age + kidslt6 + nwifeinc | 
    . - lwage + exper + expersq, data = base)

Residuals:
     Min       1Q   Median       3Q      Max 
-4570.13  -654.08   -36.94   569.86  8372.91 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept) 2225.662    574.564   3.874 0.000124 ***
lwage       1639.556    470.576   3.484 0.000545 ***
educ        -183.751     59.100  -3.109 0.002003 ** 
age           -7.806      9.378  -0.832 0.405664    
kidslt6     -198.154    182.929  -1.083 0.279325    
nwifeinc     -10.170      6.615  -1.537 0.124942    

Diagnostic tests:
                 df1 df2 statistic  p-value    
Weak instruments   2 421     9.329 0.000109 ***
Wu-Hausman         1 421    43.665 1.18e-10 ***
Sargan             1  NA     0.862 0.353123    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1354 on 422 degrees of freedom
Multiple R-Squared: -2.008,	Adjusted R-squared: -2.043 

In [ ]:
# Regresión para hours usando estimación MCO
oferta_mco <- lm(hours ~ lwage + educ + age + kidslt6 + nwifeinc, data=base)
summary(oferta_mco)


Call:
lm(formula = hours ~ lwage + educ + age + kidslt6 + nwifeinc, 
    data = base)

Residuals:
    Min      1Q  Median      3Q     Max 
-1364.8  -675.4    48.2   559.6  3588.2 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) 1523.7748   305.5755   4.987 8.99e-07 ***
lwage         -2.0468    54.8801  -0.037  0.97027    
educ          -6.6219    18.1163  -0.366  0.71491    
age            0.5623     5.1400   0.109  0.91295    
kidslt6     -328.8584   101.4573  -3.241  0.00128 ** 
nwifeinc      -5.9185     3.6833  -1.607  0.10884    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 766.6 on 422 degrees of freedom
Multiple R-squared:  0.03611,	Adjusted R-squared:  0.02469 
F-statistic: 3.162 on 5 and 422 DF,  p-value: 0.008173


In [ ]:
# Ver los resultados para la oferta
table_oferta <- mtable("MCO" = oferta_mco,
                  "MC2E (Primera Etapa)" = oferta_first,
                  "MC2E (Segunda Etapa)" = oferta_second,
            summary.stats = c("N", "R-squared"),
            coef.style = "default", 
            digits=3)
table_oferta


Calls:
MCO hours: lm(formula = hours ~ lwage + educ + age + kidslt6 + nwifeinc, 
    data = base)
MC2E (Primera Etapa) lwage: lm(formula = lwage ~ exper + expersq + educ + age + kidslt6 + 
    nwifeinc, data = base)
MC2E (Segunda Etapa) hours: ivreg(formula = hours ~ lwage + educ + age + kidslt6 + nwifeinc | 
    . - lwage + exper + expersq, data = base)

                MCO hours   MC2E (Primera Etapa) lwage  MC2E (Segunda Etapa) hours  
               -----------          ----------                 -----------          
                  hours               lwage                       hours             
------------------------------------------------------------------------------------
  (Intercept)  1523.775***           -0.447                    2225.662***          
               (305.575)             (0.285)                   (574.564)            
  lwage          -2.047                                        1639.556***          
                (54.880)                      


*  En el modelo MCO, el efecto de salario sobre horas trabajadas no es significativo.
*  En el modelo MC2E, hay un efecto significativo del salario sobre las horas trabajadas. El coeficiente de `lwage` en la ecuación de horas es 1640. Por cada 1% de aumento en el salario, las horas trabajadas aumentan en 16.40 horas.
*  La magnitud y la significancia de los coeficientes cambian usando MCO y MC2E.
*  Los coeficientes de los instrumentos $exper$ y $exper^{2}$ son individualmente significativos. Un test F muestra que estos coeficientes son conjuntamente significativos.



### **Estimación para la Demanda Laboral**

**MC2E, primera etapa**: forma reducida, regresión de variables endógenas en todas las variables exógenas y obtención de valores predichos.

**Primera etapa para la ecuación 2**: variable endógena `hours` sobre instrumentos `age`, `kidslt6`, `nwifeinc` y variables exógenas:
\begin{gather*}
hours = \delta_{0} + \delta_{1}age + \delta_{2}kidslt6 + \delta_{3}nwifeinc + \delta_{4}educ + \delta_{5}exper + \delta_{6}exper^{2} + \mu_{2}
\end{gather*}

In [ ]:
# MC2E Primera etapa
demanda_first <- lm(hours ~ age + kidslt6 + nwifeinc + educ + exper + expersq, base)
summary(demanda_first)


Call:
lm(formula = hours ~ age + kidslt6 + nwifeinc + educ + exper + 
    expersq, data = base)

Residuals:
    Min      1Q  Median      3Q     Max 
-1521.2  -556.8    63.9   448.7  3310.6 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) 1691.0312   312.4039   5.413 1.04e-07 ***
age          -15.7096     5.6871  -2.762 0.005991 ** 
kidslt6     -294.8382    96.8761  -3.043 0.002485 ** 
nwifeinc      -0.1066     3.6261  -0.029 0.976570    
educ         -17.7573    16.3887  -1.084 0.279205    
exper         51.7395    14.5003   3.568 0.000401 ***
expersq       -0.5758     0.4390  -1.312 0.190374    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 729.8 on 421 degrees of freedom
Multiple R-squared:  0.1287,	Adjusted R-squared:  0.1163 
F-statistic: 10.36 on 6 and 421 DF,  p-value: 1.014e-10


**MC2E, segunda etapa**: estimar las ecuaciones usando los valores predichos de la primera etapa para variables endógenas
\begin{gather*}
lwage = \alpha_{0} + \alpha_{1}\hat{hours} + \alpha_{2}educ + \alpha_{3}exper + \alpha_{4}exper^{2} + \epsilon_{2}
\end{gather*}

In [ ]:
# Regresion para lwage usando estimación MC2E
# hours es instrumentalizada por variables de otra ecuación
demanda_second <- ivreg(lwage ~ hours + educ + exper + expersq |
                  ~ . - hours + age + kidslt6 + nwifeinc,
                data = base)
summary(demanda_second, diagnostics = TRUE)


Call:
ivreg(formula = lwage ~ hours + educ + exper + expersq | ~. - 
    hours + age + kidslt6 + nwifeinc, data = base)

Residuals:
     Min       1Q   Median       3Q      Max 
-3.49800 -0.29307  0.03208  0.36486  2.45912 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept) -0.6557254  0.3377883  -1.941   0.0529 .  
hours        0.0001259  0.0002546   0.494   0.6212    
educ         0.1103300  0.0155244   7.107 5.08e-12 ***
exper        0.0345824  0.0194916   1.774   0.0767 .  
expersq     -0.0007058  0.0004541  -1.554   0.1209    

Diagnostic tests:
                 df1 df2 statistic p-value   
Weak instruments   3 421     4.457 0.00426 **
Wu-Hausman         1 422     0.551 0.45834   
Sargan             2  NA     2.925 0.23164   
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.6794 on 423 degrees of freedom
Multiple R-Squared: 0.1257,	Adjusted R-squared: 0.1174 
Wald test: 19.03 on 4 and 423 DF,  p-value: 2

In [ ]:
# Regresion para lwage usando estimación MCO
demanda_mco <- lm(lwage ~ hours + educ + exper + expersq, base)
summary(demanda_mco)


Call:
lm(formula = lwage ~ hours + educ + exper + expersq, data = base)

Residuals:
     Min       1Q   Median       3Q      Max 
-3.08669 -0.31040  0.06009  0.38325  2.33164 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept) -4.620e-01  2.038e-01  -2.266 0.023933 *  
hours       -5.655e-05  4.378e-05  -1.292 0.197206    
educ         1.062e-01  1.417e-02   7.496  3.9e-13 ***
exper        4.470e-02  1.339e-02   3.339 0.000914 ***
expersq     -8.585e-04  3.946e-04  -2.176 0.030144 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.6659 on 423 degrees of freedom
Multiple R-squared:  0.1601,	Adjusted R-squared:  0.1522 
F-statistic: 20.16 on 4 and 423 DF,  p-value: 3.258e-15


In [ ]:
# Ver los resultados para la demanda
table_demanda <- mtable("MCO" = demanda_mco,
                  "MC2E (Primera Etapa)" = demanda_first,
                  "MC2E (Segunda Etapa)" = demanda_second,
            summary.stats = c("N", "R-squared"),
            coef.style = "default", 
            digits=4)
table_demanda


Calls:
MCO lwage: lm(formula = lwage ~ hours + educ + exper + expersq, data = base)
MC2E (Primera Etapa) hours: lm(formula = hours ~ age + kidslt6 + nwifeinc + educ + exper + 
    expersq, data = base)
MC2E (Segunda Etapa) lwage: ivreg(formula = lwage ~ hours + educ + exper + expersq | ~. - 
    hours + age + kidslt6 + nwifeinc, data = base)

                MCO lwage   MC2E (Primera Etapa) hours  MC2E (Segunda Etapa) lwage  
               -----------         ------------                -----------          
                  lwage               hours                       lwage             
------------------------------------------------------------------------------------
  (Intercept)   -0.4620*           1691.0312***                 -0.6557             
                (0.2038)           (312.4039)                   (0.3378)            
  hours         -0.0001                                          0.0001             
                (0.0000)                                   

*  En el modelo MCO, el efecto de las horas sobre el salario no es significativo.
*  En el modelo MC2E, el efecto de horas sobre el salario tampoco no es significativo.
*  Los resultados MCO son muy similares a los resultados MC2E para ésta ecuación.
*  Dos de los tres coeficientes `age`, `kidslt6` y `nwifeinc` son individualmente significativos. Un test F muestra que estos coeficientes son conjuntamente significativos.

### **Evaluando condición de rango**

Evaluar la condición de rango implica estimar la ecuación de forma reducida y probar la significancia de las variables instrumentales


**Para la forma reducida de la oferta:**

Estimando la ecuación de la forma reducida para `lwage`:

\begin{gather*}
lwage = \gamma_{0} + \gamma_{1}exper + \gamma_{2}exper^{2} + \gamma_{3}educ + \gamma_{4}age + \gamma_{5}kidslt6 + \gamma_{6}nwifeinc + \mu_{1}
\end{gather*}

Probar si coeficientes $\gamma_{1}$ y $\gamma_2$ de $exper$ y $exper^2$ son conjuntamente significativos diferente de cero:

\begin{gather*}
H_0: \gamma_1=0 \quad y \quad \gamma_2=0 \quad \text{(ecuación para hours no es identificada)}
\end{gather*}
\begin{gather*}
H_0: \gamma_1\neq0 \quad y \quad \gamma_2\neq0 \quad \text{(ecuación para hours es identificada)}
\end{gather*}

In [ ]:
# Ecuación de la forma reducida para lwage, ecuación de identificación para hours
model1 <- lm(lwage ~ educ + age + kidslt6 + nwifeinc + exper + expersq, base)
summary(model1)
linearHypothesis(model1, c("exper = 0", "expersq = 0"))


Call:
lm(formula = lwage ~ educ + age + kidslt6 + nwifeinc + exper + 
    expersq, data = base)

Residuals:
     Min       1Q   Median       3Q      Max 
-3.09020 -0.30506  0.05005  0.37763  2.35628 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept) -0.4471607  0.2852028  -1.568  0.11766    
educ         0.1011113  0.0149618   6.758 4.68e-11 ***
age         -0.0025561  0.0051920  -0.492  0.62275    
kidslt6     -0.0532185  0.0884411  -0.602  0.54767    
nwifeinc     0.0055600  0.0033104   1.680  0.09378 .  
exper        0.0418643  0.0132377   3.162  0.00168 ** 
expersq     -0.0007625  0.0004008  -1.903  0.05779 .  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.6662 on 421 degrees of freedom
Multiple R-squared:  0.1633,	Adjusted R-squared:  0.1514 
F-statistic: 13.69 on 6 and 421 DF,  p-value: 3.158e-14


,Res.Df,RSS,Df,Sum of Sq,F,Pr(>F)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,423,195.1392,NA,NA,NA,NA
2,421,186.8577,2,8.281511,9.329333,0.0001085274


*   F estadístico=9.33 y p-value=0.0001. Los coeficientes son conjuntamente significativos.
*   La condición de rango se cumple y la ecuación para `hours` es identificada.

**Para la forma reducida de la demanda:**

Estimando la ecuación de la forma reducida para `hours`:

\begin{gather*}
hours = \delta_{0} + \delta_{1}age + \delta_{2}kidslt6 + \delta_{3}nwifeinc + \delta_{4}educ + \delta_{5}exper + \delta_{6}exper^{2} + \mu_{2}
\end{gather*}

`age`, `kidslt6` y `nwifeinc` son los instrumentos para `hours`:

Probar si coeficientes $\delta_{1}$, $\delta_{2}$ y $\delta_3$ de `age`, `kidslt6` y `nwifeinc` son conjuntamente significativos diferente de cero:

\begin{gather*}
H_0: \delta_1=0 \quad \delta_2=0 \quad \delta_3=0 \quad \text{(ecuación para lwage no es identificada)}
\end{gather*}
\begin{gather*}
H_0: \delta_1\neq0 \quad \delta_2\neq0 \quad \delta_3\neq0 \quad \text{(ecuación para lwage es identificada)}
\end{gather*}

In [ ]:
# Ecuación de la forma reducida para hours, ecuación de identificación para lwage
model2 <- lm(hours ~ educ + age + kidslt6 + nwifeinc + exper + expersq, base)
summary(model2)
linearHypothesis(model2, c("age = 0", "kidslt6 = 0", "nwifeinc = 0"))


Call:
lm(formula = hours ~ educ + age + kidslt6 + nwifeinc + exper + 
    expersq, data = base)

Residuals:
    Min      1Q  Median      3Q     Max 
-1521.2  -556.8    63.9   448.7  3310.6 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) 1691.0312   312.4039   5.413 1.04e-07 ***
educ         -17.7573    16.3887  -1.084 0.279205    
age          -15.7096     5.6871  -2.762 0.005991 ** 
kidslt6     -294.8382    96.8761  -3.043 0.002485 ** 
nwifeinc      -0.1066     3.6261  -0.029 0.976570    
exper         51.7395    14.5003   3.568 0.000401 ***
expersq       -0.5758     0.4390  -1.312 0.190374    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 729.8 on 421 degrees of freedom
Multiple R-squared:  0.1287,	Adjusted R-squared:  0.1163 
F-statistic: 10.36 on 6 and 421 DF,  p-value: 1.014e-10


,Res.Df,RSS,Df,Sum of Sq,F,Pr(>F)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,424,231321286,NA,NA,NA,NA
2,421,224200428,3,7120858,4.457145,0.004264957


*   F estadístico=4.46 y p-value=0.0043. Los coeficientes son conjuntamente significativos.
*   La condición de rango se cumple y la ecuación para `lwage` es identificada.